In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

API_URL = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {os.environ["OPENROUTER_API_KEY"]}",
    "Content-Type": "application/json"
}

RQ = "Analyze the following code snippet and identify the most prominent code smell according to established software engineering principles. If there is no discernible code smell, respond with 'None'. Then, suggest which modifications or refactorings could be applied to mitigate the identified code smell."

In [2]:
def make_question(snippet):
    return f"""{RQ}

{snippet}"""

def save_markdown(filename, content):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(content)
        print(f"Successfully saved content to '{filename}'")
    except Exception as e:
        print(f"Error saving to file: {e}")

In [3]:
code_snippets = ["""aux = x\nx = y\ny = aux"""]

for index, snippet in enumerate(code_snippets):
    try:
        print(f"Snippet {index}:")
        data = {
            "model": "deepseek/deepseek-chat:free",
            "messages": [{"role": "user", "content": make_question(snippet)}]
        }

        response = requests.post(API_URL, json=data, headers=headers)
        
        if response.status_code == 200:
            parsed_response = response.json()
            answer = ""
            for index, resp in enumerate(parsed_response["choices"]):
                content = resp["message"]["content"]
                if index == 0:
                    answer += content
                else:
                    answer += ("\n\n" + content)
            save_markdown(f"responses/snippet{index}.md", answer)
        else:
            print("Falha ao buscar dados da API do OpenRouter. Código de Status:", response.status_code)
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

Snippet 0:
Successfully saved content to 'responses/snippet0.md'
